In [4]:
# -*- coding:utf-8 -*-
import sys
import time
import serial
import threading
import queue


class Protocol(object):
    """\
    Protocol as used by the ReaderThread. This base class provides empty
    implementations of all methods.
    """

    def connection_made(self, transport):
        """Called when reader thread is started"""

    def data_received(self, data):
        """Called with snippets received from the serial port"""

    def connection_lost(self, exc):
        """\
        Called when the serial port is closed or the reader loop terminated
        otherwise.
        """
        if isinstance(exc, Exception):
            raise exc


class ReaderThread(threading.Thread):
    """\
    Implement a serial port read loop and dispatch to a Protocol instance (like
    the asyncio.Protocol) but do it with threads.
    Calls to close() will close the serial port but it is also possible to just
    stop() this thread and continue the serial port instance otherwise.
    """

    def __init__(self, serial_instance, protocol_factory):
        """\
        Initialize thread.
        Note that the serial_instance' timeout is set to one second!
        Other settings are not changed.
        """
        super(ReaderThread, self).__init__()
        self.daemon = True
        self.serial = serial_instance
        self.protocol_factory = protocol_factory
        self.alive = True
        self._lock = threading.Lock()
        self._connection_made = threading.Event()
        self.protocol = None
        self.busy = False

    def stop(self):
        """Stop the reader thread"""
        self.alive = False
        if hasattr(self.serial, "cancel_read"):
            self.serial.cancel_read()
        self.join(2)

    def run(self):
        """Reader loop"""
        if not hasattr(self.serial, "cancel_read"):
            self.serial.timeout = 1
        self.protocol = self.protocol_factory()
        try:
            self.protocol.connection_made(self)
        except Exception as e:
            self.alive = False
            self.protocol.connection_lost(e)
            self._connection_made.set()
            return
        error = None
        self._connection_made.set()
        while self.alive and self.serial.is_open:
            try:
                # read all that is there or wait for one byte (blocking)
                data = self.serial.read(self.serial.in_waiting or 1)
            except serial.SerialException as e:
                # probably some I/O problem such as disconnected USB serial
                # adapters -> exit
                error = e
                break
            else:
                if data:
                    # make a separated try-except for called used code
                    try:
                        self.protocol.data_received(data)
                    except Exception as e:
                        error = e
                        break
        self.alive = False
        self.protocol.connection_lost(error)
        self.protocol = None

    def write(self, data):
        """Thread safe writing (uses lock)"""
        with self._lock:

            print(data)
            self.serial.write(data)

    def close(self):
        """Close the serial port and exit reader thread (uses lock)"""
        # use the lock to let other threads finish writing
        with self._lock:
            # first stop reading, so that closing can be done on idle port
            self.stop()
            self.serial.close()

    def connect(self):
        """
        Wait until connection is set up and return the transport and protocol
        instances.
        """
        if self.alive:
            self._connection_made.wait()
            if not self.alive:
                raise RuntimeError("connection_lost already called")
            return (self, self.protocol)
        else:
            raise RuntimeError("already stopped")

    # - -  context manager, returns protocol

    def __enter__(self):
        """\
        Enter context handler. May raise RuntimeError in case the connection
        could not be created.
        """
        self.start()
        self._connection_made.wait()
        if not self.alive:
            raise RuntimeError("connection_lost already called")
        return self.protocol

    def __exit__(self, exc_type, exc_val, exc_tb):
        """Leave context: close port"""
        self.close()


In [12]:
# 프로토콜
class rawProtocal(Protocol):
    # 연결 시작시 발생
    def connection_made(self, transport):
        self.transport = transport
        self.running = True
        self.busy = False

    # 연결 종료시 발생
    def connection_lost(self, exc):
        self.transport = None
        self.running = False
    # 데이터가 들어오면 이곳에서 처리함.
    def data_received(self, data):
        # 입력된 데이터와 키맵을 비교해서 있다면
        print("read : ", data)
        if data == b'K' or data[-1] == b'K':
            self.busy = False

    # 데이터 보낼 때 함수
    def write(self, data):
        self.busy = True
        self.transport.write(data)

    # 종료 체크
    def isDone(self):
        return self.running

    def isBusy(self):
        return self.busy


In [14]:
# 포트 설정
PORT = 'COM17'
# 연결
ser = serial.serial_for_url(PORT,
                baudrate=115200,
				bytesize=8,
				parity='N',
				stopbits=2,
				xonxoff = 0,
				rtscts = 0,
				dsrdtr = 1,
				timeout=5)

# 쓰레드 시작
count = 0
total = 810
with ReaderThread(ser, rawProtocal) as p:
	f = open("TC66.bin", 'rb')
	while p.isDone():
		if(not p.isBusy()):
			p.write(f.read(64))
			count+=1
			print(count,"/",total)
		time.sleep(0.02)
	f.close()

b'\xbc\xb8\xa9\x11\xa6\xe1a\x13:dT+RJ_\xfe+\xd3\x8cM\xf8\r\r\xd4\xeb\xbf\xe5\x14+&\xd0o\xbf~\xb4auaE\xc1\xc7\xdbpm\x90\xd7\xcc\x1c}\xe6\xb44\x95p\xa4\x8by^p\xdb\xe5b\x06P'
1 / 810
read :  b'O'
read :  b'K'
b'\xb8\x83n\xf5H4Z$p\xaahAR\xa9\xbb\xa8\x98\xacz\x9f\xbfg\xbf\xa5?\xbf\x1a\x94\xb5\x8e\xdeK\xb8\x83n\xf5H4Z$p\xaahAR\xa9\xbb\xa8\x94\x8d\x9dw\xda\xf7o\x07u\x14\x92DG\xe1\x01\xa1'
2 / 810
read :  b'O'
read :  b'K'
b'\x98\x8a4\xf6\xa2\xe46\xe8\xab7\xa5\xb5\xba\xa6\xcb\xe9\xb8\x83n\xf5H4Z$p\xaahAR\xa9\xbb\xa8\xb8\x83n\xf5H4Z$p\xaahAR\xa9\xbb\xa8;:\xb0\xd4\x8fzi\x05\x10m\xbbT\x0b\xf3c\xbd'
3 / 810
read :  b'O'
read :  b'K'
b'q;q\x81\x04\x8c:\xb6\x99\xaf\xac\xfd\xbf\x0cjP\xa4\xd6Z\xc6+\xf1\x885\xf4\xb9\x7f\r\x19\x89c\n\x85\xfd\xec\xaf\xd4\x02_\xfd\x13\xef\xe1\x940\xe5\x7f\xd9\xb8\x83n\xf5H4Z$p\xaahAR\xa9\xbb\xa8'
4 / 810
read :  b'O'
read :  b'K'
b'`\xf8\xde \x08\x1fv\x1c\x83\x07\xc7]>\xc4s\x973\x94\xd6\xabB0\xf6\xe7\xda\xc6\xb6D\xe6+\xf8\xa73\x94\xd6\xabB0\xf6\xe7\xda\xc6\xb6D\xe6+\xf8\x